In [28]:
from random import shuffle

a = "he shall not pass you to bring your sticks of bombs and pet armadillo along"
b = "he will not allow you to bring your sticks of dynamite and pet armadillo along"

In [29]:
def shingle(text: str, k: int):
    shingle_set = []
    for i in range(len(text) - k+1):
        shingle_set.append(text[i:i+k])
    return set(shingle_set)

def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(1, len(vocab)+1))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes

def create_hash(vector: list, minhash_func: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_func:
        for i in range(1, len(vocab)+1):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

def jaccard(a: set, b: set):
    return len(a.intersection(b)) / len(a.union(b))


# ---- LSH ----
def split_vector(signature, b):
    assert len(signature) % b == 0
    r = int(len(signature) / b)
    # code splitting signature in b parts
    subvecs = []
    for i in range(0, len(signature), r):
        subvecs.append(signature[i : i+r])
    return subvecs

In [30]:
k=10

a_shingles = shingle(a, k)
b_shingles = shingle(b, k)

vocab = list(a_shingles.union(b_shingles))

a_1hot = [1 if x in a_shingles else 0 for x in vocab]
b_1hot = [1 if x in b_shingles else 0 for x in vocab]

# we create 20 minhash vectors
minhash_func = build_minhash_func(len(vocab), 20)

a_sig = create_hash(a_1hot, minhash_func)
b_sig = create_hash(b_1hot, minhash_func)



In [31]:
band_a = split_vector(a_sig, 10)
band_b = split_vector(b_sig, 10)

print(band_a)
print(band_b)

for a_rows, b_rows in zip(band_a, band_b):
    if a_rows == b_rows:
        print(f"Candidate pair: {a_rows} == {b_rows}")
        # we only need one band to match
        break

[[77, 21], [27, 67], [99, 18], [17, 43], [31, 62], [43, 13], [6, 6], [73, 59], [34, 36], [28, 36]]
[[77, 21], [27, 49], [79, 18], [51, 44], [49, 62], [10, 26], [44, 6], [56, 69], [70, 36], [9, 36]]
Candidate pair: [77, 21] == [77, 21]
